In [1]:
import threading
import time
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")
import pynq.lib.rgbled as rgbled 

led4 = rgbled.RGBLED(4)
btns = base.btns_gpio

In [ ]:
def blink(t, d, n):
    '''
    Function to blink the LEDs
    Params:
      t: number of times to blink the LED
      d: duration (in seconds) for the LED to be on/off
      n: index of the LED (0 to 4)
    '''
    if n == 4:
        for i in range(int(t/2)):
            led4.on(2)
            time.sleep(d)
            led4.off()
            time.sleep(d)
    else:
        for i in range(t):
            base.leds[n].toggle()
            time.sleep(d)
        base.leds[n].off()
        
def button_monitor_t():
    cond = True
    while cond:
        if btns.read() != 0:
            stop_event.set()
            cond = False

def philosopher_t(_l0, _l1, num):
    while not stop_event.is_set():
        # do stuff

In [ ]:
# Initialize and launch the threads
threads = []

locks = []
for i in range(5):
    locks.append(threading.Lock())

for i in range(4):
    t = threading.Thread(target=philosopher_t, args=(locks[i], locks[i+1], i))
    threads.append(t)
    t.start()

t = threading.Thread(target=philosopher_t, args=(locks[0], locks[4], i))
threads.append(t)
t.start()
    
stop_event = threading.Event()
t = threading.Thread(target=button_monitor_t)
t.start()


for t in threads:
    name = t.name
    t.join()
    print('{} joined'.format(name))